In [1]:
import numpy as np 
import pandas as pd
import os
import io
import glob
import re
import sys
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
import branca
import json

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [110]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['SeoulFloating',
 'TimeAge',
 'SearchTrend',
 'TimeProvince',
 'Weather',
 'SFedit',
 'PatientRoute',
 'PatientInfo',
 'Region',
 'TimeGender',
 'Policy',
 'Case',
 'Time']

In [3]:
rsv = pd.read_csv('custom_data/rsv.csv',header=None)
rsv = rsv.set_index([0,1])
rsv = rsv.drop(3,axis=1)
rsv.columns = ['rsv']
rsv

rsv
0    1        
2015 1   577.0
     2   519.0
     3   387.0
     4   281.0
     5   299.0
...        ...
2020 13   17.0
     14   17.0
     15   19.0
     16    8.0
     17    NaN

[278 rows x 1 columns]

In [22]:
fig3 = rsv.iplot(asFigure=True, kind='scatter')
fig3.show()

In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [5]:
decomp = seasonal_decompose(rsv[rsv.rsv.notnull()],model='additive',period=52)

In [6]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1)

fig.add_trace(go.Scatter(y=decomp.observed),
              row=1, col=1)
fig.add_trace(go.Scatter(y=decomp.resid),
              row=2, col=1)
fig.add_trace(go.Scatter(y=decomp.trend),
              row=3, col=1)
fig.add_trace(go.Scatter(y=decomp.seasonal),
              row=4, col=1)

fig.show()

In [7]:
temp = decomp.seasonal.unique()
temp  = np.roll(temp, shift= -26)
temp = pd.DataFrame(temp)

In [8]:
fig = temp.iplot(asFigure=True,kind='scatter')
fig.show()

In [20]:
len(temp)

52

In [9]:
timedat = Time.set_index('date').drop('time',axis=1)
dailydata = timedat.diff()
dailydata.iloc[0,:] = [0,0,0,0,0]

In [18]:
timedat = Time.set_index('date').drop('time',axis=1)
dailyconf = timedat.confirmed.diff()
dailyconf[0] = 1
dailyconf = pd.DataFrame(dailyconf)
dailyconf.index = pd.to_datetime(dailyconf.index)
dailyconf['week'] = dailyconf.index.week
dailyconf['year'] = pd.DatetimeIndex(Time.date).year
weekconf = dailyconf.groupby(['year','week']).sum() 

In [38]:
dailyconf.head()

,confirmed,week,year
date,,,
2020-01-20,1.0,4,2020
2020-01-21,0.0,4,2020
2020-01-22,0.0,4,2020
2020-01-23,0.0,4,2020
2020-01-24,1.0,4,2020


In [19]:
fig2 = weekconf.iplot(asFigure=True, kind='scatter')
fig2.show()

In [21]:
len(weekconf)

14

rsv 의 한 주기의 길이는 52주로 일정

2017말 - 2018초 경을 보면 최저점에서 약 23주만에 최고점을 찍은 뒤 약 30주에 걸려 최저점을 찍고 다시 올라감

그렇다면 코로나도 비슷하지 않을까?

현재 코로나 시작한지 14주 됐는데 벌써 주기의 마지막처럼 보임

첫 발병 후 7주만에 최고점이었음 

단순히 비율로만 생각해보면 최저점은 약 9.13주 뒤 = 약 64일 뒤임

즉 4월 말로부터 2.13주동안 더 하락할 것임 최저점은 5월 중순

이를 유지하다가 다시 올 팬데믹은 5월 중순에서 7주 뒤인 7월 말임

근데 너무 많은 무언가를 무시한 단순계산결과라 찜찜맨

In [25]:
Weather.head()

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
0,10000,Seoul,2016-01-01,1.2,-3.3,4.0,0.0,3.5,90.0,73.0
1,11000,Busan,2016-01-01,5.3,1.1,10.9,0.0,7.4,340.0,52.1
2,12000,Daegu,2016-01-01,1.7,-4.0,8.0,0.0,3.7,270.0,70.5
3,13000,Gwangju,2016-01-01,3.2,-1.5,8.1,0.0,2.7,230.0,73.1
4,14000,Incheon,2016-01-01,3.1,-0.4,5.7,0.0,5.3,180.0,83.9


In [27]:
wthr_reindex = Weather.set_index('date')

In [35]:
current_wthr = wthr_reindex[wthr_reindex.index.isin(Time.date)]

In [50]:
fig2 = current_wthr.precipitation.iplot(asFigure=True, kind='bar')
fig2.add_trace(go.Scatter(x = dailyconf.index , y=dailyconf.confirmed))
fig2.show()

강수량이 국소적인 하락에는 영향이 좀 있는 것 같음

# 20대에 대하여..

In [72]:
patientcount = PatientInfo.groupby('age').count().patient_id

In [73]:
fig = patientcount.iplot(asFigure=True, kind='bar')
fig.show()

혹시 우리나라에 20대가 제일 많은가?

In [78]:
population = pd.read_csv('custom_data/korea_2020_pop.csv',encoding='CP949')

In [80]:
population.columns = ['region','total','total2','0s','10s','20s','30s','40s','50s','60s','70s','80s','90s','100s']

In [86]:
totalpop = population.iloc[0,[3,4,5,6,7,8,9,10,11,12,13]]

In [87]:
fig = totalpop.iplot(asFigure=True, kind='bar')
fig.show()

? 50대가 제일 많음.. 대체왜..

In [42]:
SeoulFloating['hour'] = pd.TimedeltaIndex(SeoulFloating.hour,unit='H')
SeoulFloating['datehour'] = pd.to_datetime(SeoulFloating['date'] )+ SeoulFloating['hour']
SeoulFloating[SeoulFloating.datehour == '2020-02-22 23:00'].fp_num
temp = SeoulFloating[SeoulFloating.date == '2020-02-23'].fp_num/2
SFedit = SeoulFloating
SFedit.loc[SFedit.date == '2020-02-23','fp_num'] = temp
temp2 = SeoulFloating[SeoulFloating.datehour == '2020-02-23 11:00:00'].fp_num *2
SFedit.loc[SFedit.datehour == '2020-02-23 11:00:00','fp_num'] = temp2

In [44]:
SFedit.to_csv('SFedit.csv')

In [52]:
birthsum = SFedit.groupby('birth_year').sum()

In [53]:
fig = birthsum.iplot(asFigure=True, kind='bar')
fig.show()

유동인구는 오히려 30,40대가 20대보다 더 많음. 근데 왜 20대의 감염률이 월등히 높을까

In [59]:
agecontact = PatientInfo.groupby('age').mean().contact_number

In [60]:
fig = agecontact.iplot(asFigure=True, kind='bar')
fig.show()

평균 접촉자수도 20대가 높은 편이 전혀아님

In [98]:
twtregion = population.loc[:,['region','20s']]
twtregion = twtregion.drop(0)
twtregion = twtregion.set_index('region')

In [99]:
fig = twtregion.iplot(asFigure=True,kind='bar')
fig.show()

지역별 20대 인구. 대구가 전혀 20대가 많은 편이 아니다. 경기도가 엄청나게 많네.

In [111]:
TP = TimeProvince[TimeProvince.date=='2020-04-20'].drop(['released','deceased','time','date'],axis=1)
regionconf = TP.set_index('province')

In [113]:
fig = regionconf.iplot(asFigure=True,kind='bar')
fig.show()

지역별 확진자 인구. 역시나 대구가 제일 많고, 경상북도 다음에 경기도 다음에 서울인듯

In [118]:
twtinfo = PatientInfo[PatientInfo.age == '20s']

In [124]:
twtinforeg = twtinfo.groupby('province').count().patient_id

In [125]:
fig = twtinforeg.iplot(asFigure=True,kind='bar')
fig.show()

위는 지역별 20대 확진자 수.경상북도에 뭐가 있었길래 20대 확진자가 이렇게나 많은거냐....... 2위인 서울의 거의 두배는 돼보인다.